### Read a set of CSV files

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/utils"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType 

In [0]:
lap_time_schema = StructType([StructField("raceId", IntegerType(), False), 
                              StructField("driverId", IntegerType(), True),
                              StructField("lap", IntegerType(), True),
                              StructField("position", IntegerType(), True),
                              StructField("time", StringType(), True),
                              StructField("milliseconds", IntegerType(), True),
                            ])


lap_times_df = spark.read \
    .schema(lap_time_schema) \
    .csv(f"{RAW_FOLDER_PATH}/lap_times") # read the whole folder   


In [0]:
lap_times_df.printSchema()

In [0]:
display(lap_times_df)

In [0]:
lap_times_df.count()

### rename columns and add new column

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, date_trunc, from_utc_timestamp

In [0]:
final_lap_times_df = add_ingestion_date(lap_times_df.withColumnRenamed("driverId", "driver_id") \
                                .withColumnRenamed("raceId", "race_id"))



display(final_lap_times_df)

### write to parquet file

In [0]:
final_lap_times_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.lap_times")



In [0]:
spark.read.parquet(f"{PROCESSED_FOLDER_PATH}/lap_times").display()

In [0]:
dbutils.notebook.exit("Success")